In [1]:
from pyomo.environ import *
from pyomo.dae import *
import numpy as np
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *

## Measurements class

In [12]:
# generate a set of measurements with extra index CA, CB, CC
# each extra index has different measure time points
t_measure_ca = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
t_measure_cb = [0, 0.25, 0.5, 0.75, 1]
t_measure_cc = [0, 0.125, 0.375, 0.625, 0.875, 1]
measure_pass = {'C':{"CA": t_measure_ca, "CB": t_measure_cb, "CC": t_measure_cc}}
var = {'C':{'CA': 5, 'CB': 2, 'CC': 1}}

In [13]:
measure_class = Measurements(measure_pass, variance=var)


C
CA
<class 'str'>
C
CB
<class 'str'>
C
CC
<class 'str'>
All measurements are flattened.
Flatten measurement name: ['C_index_CA', 'C_index_CB', 'C_index_CC']
Flatten measurement variance: {'C_index_CA': 5, 'C_index_CB': 2, 'C_index_CC': 1}


In [18]:
print(measure_class.flatten_measure_name)
print(measure_class.flatten_variance)
print(measure_class.flatten_measure_timeset['C_index_CB'])

['C_index_CA', 'C_index_CB', 'C_index_CC']
{'C_index_CA': 5, 'C_index_CB': 2, 'C_index_CC': 1}
[0, 0.25, 0.5, 0.75, 1]


In [19]:
# a legal subset
subset_1 = {'C':{'CA': t_measure_cb}}
measure_subclass1 = Measurements(subset_1)
measure_class.check_subset(measure_subclass1)

# an illegal subset with wrong time
# Expect error: ('The time of ', 0.125, ' is not included as measurements before.')
#subset_2 = {'C':{"CB": t_measure_cc}}
#measure_subclass2 = Measurements(subset_2)
#dd = measure_class.check_subset(measure_subclass2)

All measurements are flattened.
Flatten measurement name: ['C_index_CA']
Flatten measurement variance: {'C_index_CA': 1}


True

## Scenario class

In [28]:
param_dict = {'A1': 84.79, 'A2': 371.72, 'E1': 7.78, 'E2': 15.05}

scenario_gene = Scenario_generator(param_dict, formula='central', step=0.1)
simultaneous_set = scenario_gene.simultaneous_scenario()
#print(simultaneous_set)
print(simultaneous_set['A1'])
print(simultaneous_set['jac-index'])
print(simultaneous_set['scena-name'])
print(simultaneous_set['eps-abs'])

{0: 93.26900000000002, 1: 84.79, 2: 84.79, 3: 84.79, 4: 76.311, 5: 84.79, 6: 84.79, 7: 84.79}
{'A1': [0, 4], 'A2': [1, 5], 'E1': [2, 6], 'E2': [3, 7]}
[0, 1, 2, 3, 4, 5, 6, 7]
{'A1': 16.958000000000002, 'A2': 74.34400000000001, 'E1': 1.556, 'E2': 3.0100000000000002}


In [32]:
scenario_gene.generate_sequential_para()
print(scenario_gene.eps_abs)
print(scenario_gene.scenario_para)

for i in range(8):
    ss_next = scenario_gene.next_sequential_scenario(i)

print(scenario_gene.next_sequential_scenario(2))

{'A1': 16.958000000000002, 'A2': 74.34400000000001, 'E1': 1.556, 'E2': 3.0100000000000002}
{'A1': [0, 4], 'A2': [1, 5], 'E1': [2, 6], 'E2': [3, 7]}
{'A1': {0: 84.79}, 'A2': {0: 371.72}, 'E1': {0: 8.558000000000002}, 'E2': {0: 15.05}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 8.479000000000001, 'A2': 37.172000000000004, 'E1': 0.8558000000000002, 'E2': 1.5050000000000001}, 'scena-name': [0]}
